In [11]:
# Combined script containing all code from the three notebooks

# --- Begin query_pinecone (2) Notebook ---

from sentence_transformers import SentenceTransformer
#connect to an existing index

from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_3StWu1_F4vFjfS4BMLmpvfuBE6aUyYE1LRVGetymK5Q4FHRBCFNbJWK2KRjtn1insArDth") # "New API key as of 4/18/2025"

index = pc.Index(name="test-index", host="https://test-index-9l7icfb.svc.aped-4627-b74a.pinecone.io")  # the name you gave it

#creating a new index

#  Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="pcsk_3StWu1_F4vFjfS4BMLmpvfuBE6aUyYE1LRVGetymK5Q4FHRBCFNbJWK2KRjtn1insArDth") # "New API key as of 4/18/2025"

# Create a dense index with integrated embedding
index_name = "test-index"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

pc.describe_index(name=index_name)

test_index = pc.Index(name="test-index", host="https://test-index-mzzv42s.svc.aped-4627-b74a.pinecone.io")  # the name you gave it


In [13]:
import pandas as pd

# Load the new, larger CSV
clean_lyrics = pd.read_csv(r"song_lyrics (4).csv")

# Quick sanity check
print("Total songs:", len(clean_lyrics))
display(clean_lyrics.head())


Total songs: 12500


,Track Name,Artist Name,Genre,Year,Lyrics,Spotify Genres,Release Date,Popularity,Explicit
0,Versace on the Floor,Bruno Mars,"dance pop,pop",11/17/2016,Lets take our time tonight girl Above us all ...,NaN,11/17/2016,78,False
1,If I Never See Your Face Again Remix,Maroon 5,"pop,barbadian pop,pop,urban contemporary",6/2/2008,Now as the summer fades I let you slip away Y...,pop,6/2/2008,49,False
2,When I Was Young,The Animals,"british invasion,classic rock,folk rock",12/2/2005,The rooms were so much colder then My fathe...,"psychedelic rock, acid rock",6/11/1991,32,False
3,Dizzy,Tommy Roe,"bubblegum pop,classic garage rock,merseybeat,r...",1/1/1977,Dizzy Im so dizzy my head is spinning Like a w...,bubblegum pop,1/1/2001,50,False
4,When Youre Gone,Bryan Adams,"canadian pop,canadian singer-songwriter,classi...",1/1/1998,When youre gone Ive been wandering around t...,NaN,1/1/1998,67,False


In [15]:
df = clean_lyrics.copy()
df['_id'] = 'rec' + df.index.astype(str)

# Peek first few IDs
print(df['_id'].head(10))


0    rec0
1    rec1
2    rec2
3    rec3
4    rec4
5    rec5
6    rec6
7    rec7
8    rec8
9    rec9
Name: _id, dtype: object


In [17]:
import re

def parse_year_field(s):
    if pd.isna(s): return pd.NaT
    s = str(s).strip()
    if re.fullmatch(r"\d{4}", s):
        return pd.to_datetime(f"{s}-01-01")
    if re.fullmatch(r"\d{4}-\d{1,2}", s):
        year, month = s.split("-")
        return pd.to_datetime(f"{year}-{month.zfill(2)}-01")
    return pd.to_datetime(s, infer_datetime_format=True, errors="coerce")

df['Year'] = df['Year'].apply(parse_year_field)
df['Year'] = df['Year'].dt.strftime('%Y-%m-%d')
display(df[['Year']].head())


/var/folders/px/kgbj015s0mx1vfbs8ptpm0l80000gn/T/ipykernel_72465/1543485848.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, infer_datetime_format=True, errors="coerce")


,Year
0,2016-11-17
1,2008-06-02
2,2005-12-02
3,1977-01-01
4,1998-01-01


In [19]:
df.to_csv('v2_cleaned_song_lyrics.csv', index=False)
df = pd.read_csv("v2_cleaned_song_lyrics.csv")
print("Reloaded cleaned file, rows:", len(df))


Reloaded cleaned file, rows: 12500


In [21]:
records = (
    df[['_id', 'Lyrics', 'Genre', 'Year', 'Track Name', 'Artist Name']]
      .rename(columns={
         'Lyrics': 'chunk_text',
         'Genre' : 'genre',
         'Year'  : 'year',
         'Track Name'  : 'track_name',
         'Artist Name' : 'artist_name'
      })
    .to_dict(orient='records')
)

print("Prepared", len(records), "records for upsert.")


Prepared 12500 records for upsert.


In [23]:

from sentence_transformers import SentenceTransformer

# 1) Embed locally
model = SentenceTransformer("all-MiniLM-L6-v2")
for rec in records:
    rec["values"] = model.encode(rec["chunk_text"]).tolist()

# 2) Batch upsert raw vectors
batch_size = 100
for i in range(0, len(records), batch_size):
    batch = records[i : i + batch_size]
    upserts = [
        {
          "id": r["_id"],
          "values": r["values"],
          "metadata": {
            "genre":      r["genre"],
            "year":       r["year"],
            "track_name": r["track_name"],
            "artist_name":r["artist_name"]
          }
        }
        for r in batch
    ]
    test_index.upsert(vectors=upserts)
    print(f"Upserted {i + len(batch)}/{len(records)} records")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

# now ready for TF-IDF, vector parsing, buckets, etc.


In [ ]:

# Load metadata‐rich subset
df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df["MetaGenres_list"] = df["MetaGenres"].apply(literal_eval)
has_meta = df[df["MetaGenres_list"].apply(lambda L: L != ["OtherMeta"])]

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit a TF–IDF vectorizer (unigrams only, English stop‐words)
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english",
    ngram_range=(1,1)
)
X_tfidf = vectorizer.fit_transform(has_meta["Lyrics"].astype(str))
terms = vectorizer.get_feature_names_out()

print(f"TF–IDF matrix shape: {X_tfidf.shape}")

In [ ]:
top_terms = {}
for bucket in sorted(has_meta["Bucket"].unique()):
    mask = has_meta["Bucket"] == bucket
    # mean tf–idf score of each term within that bucket
    mean_tfidf = X_tfidf[mask].mean(axis=0).A1
    top20_idx = np.argsort(mean_tfidf)[-20:][::-1]
    top_terms[bucket] = [terms[i] for i in top20_idx]

# Peek at the first 10 terms for each bucket
for bucket, words in top_terms.items():
    print(f"{bucket:12s} → {', '.join(words[:10])} …")


In [ ]:
import pandas as pd

themes = ["love", "party", "breakup"]
all_hits = []

for theme in themes:
    res = test_index.search(
        namespace="test-namespace",
        query={"inputs": {"text": theme}, "top_k": 20},
        fields=["chunk_text"],
        rerank={
            "model":"bge-reranker-v2-m3",
            "top_n":5,
            "rank_fields":["chunk_text"]
        }
    )
    for m in res["matches"]:
        all_hits.append({
            "theme": theme,
            "song_id": m["id"],
            "score": m["score"]
        })

df_themes = pd.DataFrame(all_hits)
print(df_themes.groupby("theme")["score"].describe())

In [ ]:
search_with_text = test_index.search(
    namespace="test-namespace",
    query={"inputs": {"text": "Dancing Clowns"}, "top_k": 4},
    fields=["category", "chunk_text"],
    rerank={
        "model": "bge-reranker-v2-m3",
        "top_n": 2,
        "rank_fields": ["chunk_text"]
    }
)

print(search_with_text)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1) Load & prepare data
clean_lyrics = pd.read_csv("v2_cleaned_song_lyrics.csv")
clean_lyrics["Year"] = pd.to_datetime(clean_lyrics["Year"], errors="coerce")
clean_lyrics = clean_lyrics.dropna(subset=["Year"])
clean_lyrics["Decade"] = (clean_lyrics["Year"].dt.year // 10 * 10).astype(str) + "s"


In [ ]:
# 2) Generate SBERT embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
vectors = model.encode(clean_lyrics["Lyrics"].astype(str).tolist(), show_progress_bar=True)
clean_lyrics["vector"] = vectors.tolist()


In [ ]:
# 3) Define similarity function
def avg_similarity(vs):
    sim = cosine_similarity(vs)
    ut = sim[np.triu_indices(len(sim), k=1)]
    return ut.mean() if len(ut)>0 else 0

# Compute per-decade
decade_similarities = {}
for dec in sorted(clean_lyrics["Decade"].unique()):
    vecs = clean_lyrics[clean_lyrics["Decade"]==dec]["vector"].tolist()
    if len(vecs)>=2:
        decade_similarities[dec] = avg_similarity(vecs)


In [ ]:
# 4) Report
for dec, sim in decade_similarities.items():
    print(f"{dec}: Avg Similarity = {sim:.4f}")


In [ ]:
import pandas as pd

# 1) Load full CSV
df = pd.read_csv("song_lyrics (4).csv")

# 2) Parse Release Date
df["Release Date"] = pd.to_datetime(df["Release Date"], errors="coerce")

print("Total rows:", len(df))
print("Parsed Release Dates:", df["Release Date"].notna().sum())


In [ ]:
# 3) Extract Year & Decade
df["Year"] = df["Release Date"].dt.year
df["Decade"] = (df["Year"] // 10 * 10).astype(pd.Int64Dtype()).astype(str) + "s"

# 4) Mark unparsable as "Unknown"
df.loc[df["Release Date"].isna(), "Decade"] = "Unknown"

# 5) View counts
print(df["Decade"].value_counts(dropna=False).sort_index())


In [ ]:
# 6) See what raw strings failed to parse
raw = pd.read_csv("song_lyrics (4).csv")
raw["Release Date_raw"] = raw["Release Date"].astype(str)
bad = raw[pd.to_datetime(raw["Release Date"], errors="coerce").isna()]
print("Sample bad raws:", bad["Release Date_raw"].unique()[:20])

# 7) Extract first 4-digit year & rebuild Decade
df["Year_extracted"] = (
    raw["Release Date_raw"]
      .str.extract(r"(\d{4})")[0]
      .astype(float)
)
df["Decade"] = (df["Year_extracted"].dropna().astype(int)//10*10)
df["Decade"] = df["Decade"].astype(pd.Int64Dtype()).astype(str)+"s"
df.loc[df["Year_extracted"].isna(), "Decade"] = "Unknown"

print(df["Decade"].value_counts().sort_index())


In [27]:
import pandas as pd
import numpy as np

# Load full CSV
df = pd.read_csv("song_lyrics (4).csv")


In [ ]:
# Parse Release Date into datetime (invalid→NaT)
df["Release Date"] = pd.to_datetime(df["Release Date"], errors="coerce")

# Extract Year & Decade, defaulting to “Unknown” if missing
df["Year"] = df["Release Date"].dt.year
df["Decade"] = df["Release Date"].apply(
    lambda ts: f"{(ts.year//10*10)}s" if pd.notna(ts) else "Unknown"
)


In [ ]:
# Split comma-delimited Spotify Genres into lists (["OtherMeta"] if empty)
df["MetaGenres"] = (
    df["Spotify Genres"].fillna("")
      .str.split(",")
      .apply(lambda L: [g.strip() for g in L if g.strip()] or ["OtherMeta"])
)


In [ ]:
def assign_meta_bucket(genres):
    # genres is a list of strings
    buckets = [map_bucket(g) for g in genres]
    # take the most common bucket among them
    return pd.Series(buckets).mode()[0]


In [ ]:
# Split into “has metadata” vs “no metadata”
has_meta = df[df["MetaGenres"].apply(lambda L: L != ["OtherMeta"])].copy()
no_meta  = df[df["MetaGenres"].apply(lambda L: L == ["OtherMeta"])].copy()

# Assign Bucket for those with metadata
has_meta["Bucket"] = has_meta["MetaGenres"].apply(assign_meta_bucket)

# Quick sanity check
print("Buckets in has_meta:\n", has_meta["Bucket"].value_counts())


In [ ]:
from sklearn.neighbors import NearestNeighbors

# 1) Fit k-NN on known songs’ SBERT vectors
X_meta = np.stack(has_meta["vector"].tolist())
nn     = NearestNeighbors(n_neighbors=5, metric="cosine").fit(X_meta)

# 2) Find neighbors for the missing-metadata songs
X_no = np.stack(no_meta["vector"].tolist())
_, idxs = nn.kneighbors(X_no)

# 3) Predict each missing one as the modal bucket of its neighbors
preds = []
for neigh_idxs in idxs:
    neigh_buckets = has_meta.iloc[neigh_idxs]["Bucket"]
    preds.append(neigh_buckets.mode()[0])

no_meta["Bucket"] = preds


In [ ]:
# Merge the two back into the main DataFrame
df.loc[has_meta.index, "Bucket"] = has_meta["Bucket"]
df.loc[ no_meta.index, "Bucket"] = no_meta["Bucket"]

# 1) No more missing buckets
print("Missing buckets:", df["Bucket"].isna().sum())

# 2) Overall bucket distribution
print("Bucket counts:\n", df["Bucket"].value_counts())

# 3) key columns
print(
    df[["Track Name","Artist Name","MetaGenres","Bucket","Decade"]]
      .head(10)
)

In [ ]:
import pandas as pd
from IPython.display import FileLink, display

# Assuming `df` is your merged DataFrame with 'Bucket'
df.to_csv("F-all_songs_merged_buckets.csv", index=False)
display(FileLink("F-all_songs_merged_buckets.csv"))


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_merged_buckets.csv")
display(df.head())
print("Songs per bucket:\n", df["Bucket"].value_counts().sort_values(ascending=False))


In [ ]:
# 1) Subset “Other”
other = df[df["Bucket"] == "Other"].copy()

# 2) Count granular MetaGenres
print("Top granular genres in Other:")
print(other["MetaGenres"]
      .explode()
      .value_counts()
      .head(20))

# 3) Sample a few tracks
display(other[["Track Name","Artist Name","MetaGenres","Decade"]]
        .sample(10, random_state=42))


In [ ]:
# Update BUCKET_MAP dict for finer genres:
BUCKET_MAP["Pop"]         += ["indie"]  
BUCKET_MAP["Rock"]        += ["rock","glam metal","alternative metal","nu metal"]  
BUCKET_MAP["Electronic"]  += ["hi-nrg"]  
BUCKET_MAP["R&B/Soul"]    += ["doo-wop","motown"]  
BUCKET_MAP["Country/Folk"]+= ["singer-songwriter"]  
BUCKET_MAP["Jazz/Blues"]  += ["adult standards"]  
BUCKET_MAP["Latin/World"] += ["hindi pop","desi","bollywood","desi pop",
                               "hindi indie","variété française","reggae","schlager"]  
BUCKET_MAP["Other"]       += ["christmas"]


In [ ]:
# 1) Re-assign buckets for has_meta
has_meta = df[df["MetaGenres"].apply(lambda L: L!=["OtherMeta"])].copy()
has_meta["Bucket"] = has_meta["MetaGenres"].apply(assign_meta_bucket)

# 2) Merge back into df
df.loc[has_meta.index, "Bucket"] = has_meta["Bucket"]

# 3) Save & link
df.to_csv("F-all_songs_refined_buckets.csv", index=False)
from IPython.display import FileLink
display(FileLink("F-all_songs_refined_buckets.csv"))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

df = pd.read_csv("F-all_songs_refined_buckets.csv")

# If PC1/PC2 missing, compute from vector column
if not {"PC1","PC2"}.issubset(df):
    vectors = df["vector"].apply(lambda v: eval(v) if isinstance(v,str) else v).tolist()
    X = np.stack(vectors)
    pca = PCA(n_components=2, random_state=42)
    coords = pca.fit_transform(X)
    df["PC1"], df["PC2"] = coords[:,0], coords[:,1]

# Save & link
df.to_csv("F-all_songs_refined_with_PCA.csv", index=False)
from IPython.display import FileLink, display
display(FileLink("F-all_songs_refined_with_PCA.csv"))


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_refined_with_PCA.csv")
other = df[df["Bucket"] == "Other"]

print("Total ‘Other’ songs:", len(other))
display(other[["Track Name","Artist Name","MetaGenres","PC1","PC2"]].head(10))


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_refined_with_PCA.csv")
print("Bucket counts:\n", df["Bucket"].value_counts().sort_values(ascending=False))

# Parse Release Date and build Decade label
df["Release Date"] = pd.to_datetime(df["Release Date"], errors="coerce")
def make_decade(ts):
    return f"{(ts.year//10*10)}s" if pd.notna(ts) else "Unknown"

df["Decade"] = df["Release Date"].apply(make_decade)

print("\nDecade counts:\n", df["Decade"].value_counts().sort_index())


In [ ]:
import pandas as pd
from IPython.display import FileLink, display

df = pd.read_csv("F-all_songs_refined_with_PCA.csv")
# Extract first 4-digit year from string, convert→float
df["Year_extracted"] = (
    df["Release Date"].astype(str)
      .str.extract(r"(\d{4})")[0]
      .astype(float)
)
# Build Decade from extracted year
df["Decade"] = df["Year_extracted"].apply(
    lambda y: f"{int(y//10*10)}s" if not pd.isna(y) else "Unknown"
)

out = "F-all_songs_with_regex_decade.csv"
df.to_csv(out, index=False)
display(FileLink(out))


In [ ]:
import pandas as pd
from IPython.display import FileLink, display

df = pd.read_csv("F-all_songs_with_regex_decade.csv")

balanced_sets = {
    "F-songs_balanced_50s_2020s.csv": ['1950s','1960s','1970s','1980s','1990s','2000s','2010s','2020s'],
    "F-songs_balanced_60s_2020s.csv": ['1960s','1970s','1980s','1990s','2000s','2010s','2020s'],
    "F-songs_balanced_70s_2020s.csv": ['1970s','1980s','1990s','2000s','2010s','2020s'],
    "F-songs_balanced_70s_2010s.csv": ['1970s','1980s','1990s','2000s','2010s']
}

for out_file, decades in balanced_sets.items():
    subset = df[df["Decade"].isin(decades)]
    n = subset["Decade"].value_counts().min()
    balanced = subset.groupby("Decade").sample(n=n, random_state=42)
    balanced.to_csv(out_file, index=False)
    display(FileLink(out_file))


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_with_regex_decade.csv")

decades = ['1950s','1960s','1970s','1980s','1990s','2000s','2010s','2020s']
slice_50_20 = df[df.Decade.isin(decades)]

min_count = slice_50_20.Decade.value_counts().min()

balanced_50_20 = (
    slice_50_20
      .groupby("Decade", group_keys=False)
      .apply(lambda grp: grp.sample(n=min_count, random_state=42))
)

print(balanced_50_20.Decade.value_counts().sort_index())


In [ ]:
import pandas as pd

files = [
    "F-songs_balanced_50s_2020s.csv",
    "F-songs_balanced_60s_2020s.csv",
    "F-songs_balanced_70s_2020s.csv",
    "F-songs_balanced_70s_2010s.csv"
]

for fn in files:
    df_bal = pd.read_csv(fn)
    counts = df_bal["Decade"].value_counts().sort_index()
    print(f"\n**{fn}**")
    print(counts)


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_with_regex_decade.csv")

hiphop_60s = df[
    (df["Decade"] == "1960s") &
    (df["Bucket"] == "Hip-Hop/Rap")
]

print(len(hiphop_60s))
print(hiphop_60s[["Track Name","Artist Name","Release Date"]].head(10))


In [ ]:
import pandas as pd

df = pd.read_csv("F-all_songs_with_regex_decade.csv")

df = df[df["Decade"].astype(str).str[:-1].astype(int) >= 1970]

df["HipHop_Source"] = df.apply(
    lambda r: "metadata" 
              if "hip hop" in [g.lower() for g in r["MetaGenres"]]
              else "inferred",
    axis=1
)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# assumes has_meta contains songs with real metadata and 'vector' & 'Bucket' columns
X = np.stack(has_meta["vector"].tolist())
y = has_meta["Bucket"].values

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# KNN training
knn = KNeighborsClassifier(n_neighbors=5, metric="cosine")
knn.fit(X_train, y_train)

# evaluation
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))
cm = confusion_matrix(y_test, y_pred, labels=knn.classes_)
print(pd.DataFrame(cm, index=knn.classes_, columns=knn.classes_))


In [ ]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df["MetaGenres_list"] = df["MetaGenres"].apply(literal_eval)
has_meta = df[df["MetaGenres_list"].apply(lambda L: L != ["OtherMeta"])].copy()

# feature matrix and labels
X = np.stack(has_meta["vector"].apply(literal_eval).tolist())
y = has_meta["Bucket"].values

print(f"Loaded {X.shape[0]} songs with {X.shape[1]}-dim vectors")


In [ ]:
from sklearn.model_selection import cross_val_score

# cross-validate k values
results = []
for k in [1, 3, 5, 7, 9, 11]:
    knn = KNeighborsClassifier(n_neighbors=k, metric="cosine")
    scores = cross_val_score(knn, X, y, cv=5, scoring="accuracy")
    results.append({"k": k, "accuracy": scores.mean()})

print(pd.DataFrame(results))


In [ ]:
# hold-out evaluation for chosen k
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
best_k = 5
knn = KNeighborsClassifier(n_neighbors=best_k, metric="cosine")
knn.fit(X_tr, y_tr)
print(classification_report(y_te, knn.predict(X_te), zero_division=0))


In [ ]:
# weighted k-NN cross-validation
knn = KNeighborsClassifier(n_neighbors=5, metric="cosine", weights="distance")
scores = cross_val_score(knn, X, y, cv=5, scoring="accuracy")
print(f"{scores.mean():.4f} ± {scores.std():.4f}")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# TF-IDF + Logistic Regression
clf = make_pipeline(
    TfidfVectorizer(max_features=5000, stop_words="english"),
    LogisticRegression(max_iter=300, class_weight="balanced")
)
scores = cross_val_score(
    clf,
    has_meta["Lyrics"].astype(str),
    has_meta["Bucket"],
    cv=5,
    scoring="accuracy"
)
print(f"{scores.mean():.4f} ± {scores.std():.4f}")


In [ ]:
from scipy import sparse
from sklearn.svm import LinearSVC

# TF-IDF + embeddings + SVC
tfidf = TfidfVectorizer(max_features=3000, stop_words="english")
X_tfidf = tfidf.fit_transform(has_meta["Lyrics"].astype(str))
X_emb = sparse.csr_matrix(X)
X_combined = sparse.hstack([X_tfidf, X_emb])

svc = LinearSVC(class_weight="balanced", max_iter=5000)
scores = cross_val_score(svc, X_combined, y, cv=5, scoring="accuracy")
print(f"{scores.mean():.4f} ± {scores.std():.4f}")


In [ ]:
# 1) Load data and parse vectors
import pandas as pd
from ast import literal_eval

df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df["vector"] = df["vector"].apply(literal_eval)


In [ ]:
# 2) Filter metadata-rich songs and prepare features
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack

has_meta = df[df["MetaGenres"].apply(lambda L: L != ["OtherMeta"])].copy()
y = has_meta["Bucket"].values

tfidf = TfidfVectorizer(max_features=3000, stop_words="english", ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(has_meta["Lyrics"].astype(str))
X_emb = csr_matrix(np.stack(has_meta["vector"].tolist()))

X_hybrid = hstack([X_tfidf, X_emb])


In [ ]:
# 3) Evaluate hybrid SVM with cross-validation and hold-out
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

svc = LinearSVC(class_weight="balanced", max_iter=5000)
cv_scores = cross_val_score(svc, X_hybrid, y, cv=5, scoring="accuracy")
print(f"CV accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

X_tr, X_te, y_tr, y_te = train_test_split(X_hybrid, y, test_size=0.2, random_state=42, stratify=y)
svc.fit(X_tr, y_tr)
y_pred = svc.predict(X_te)
print(classification_report(y_te, y_pred, zero_division=0))


In [ ]:
# 4) Load PCA-refined file and parse vectors
import pandas as pd

df_pca = pd.read_csv("F-all_songs_refined_with_PCA.csv")
df_pca["vector"] = df_pca["vector"].apply(literal_eval)

if "Decade" not in df_pca.columns:
    df_pca["Year_int"] = pd.to_datetime(df_pca["Year"], errors="coerce").dt.year
    df_pca["Decade"] = (df_pca["Year_int"] // 10 * 10).astype("Int64").astype(str) + "s"
    df_pca.drop(columns=["Year_int"], inplace=True)

has_meta_pca = df_pca[df_pca["MetaGenres"].apply(lambda L: L != ["OtherMeta"])].copy()
no_meta_pca  = df_pca[df_pca["MetaGenres"].apply(lambda L: L == ["OtherMeta"])].copy()


In [ ]:
# 5) Retrain hybrid SVM on metadata-rich PCA data
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import LinearSVC

tfidf2 = TfidfVectorizer(max_features=3000, stop_words="english", ngram_range=(1,2))
X_tfidf2 = tfidf2.fit_transform(has_meta_pca["Lyrics"].astype(str))
X_emb2 = csr_matrix(np.stack(has_meta_pca["vector"].tolist()))
X_hybrid2 = hstack([X_tfidf2, X_emb2])

y2 = has_meta_pca["Bucket"].values
svc2 = LinearSVC(class_weight="balanced", max_iter=5000)
svc2.fit(X_hybrid2, y2)


In [ ]:
# 6) Predict buckets for songs without metadata
X_tfidf_no = tfidf2.transform(no_meta_pca["Lyrics"].astype(str))
X_emb_no = csr_matrix(np.stack(no_meta_pca["vector"].tolist()))
X_hybrid_no = hstack([X_tfidf_no, X_emb_no])

preds_no = svc2.predict(X_hybrid_no)
no_meta_pca["Bucket"] = preds_no

df_pca.loc[no_meta_pca.index, "Bucket"] = no_meta_pca["Bucket"]
df_pca.to_csv("F-all_songs_raw_with_buckets.csv", index=False)


In [ ]:
# 7) Extract year from Release Date and assign Decade
import numpy as np
import re

df_final = pd.read_csv("F-all_songs_refined_with_PCA.csv")

def extract_year(s):
    m = re.search(r"(\d{4})", str(s))
    return int(m.group(1)) if m else np.nan

df_final["Year_Extracted"] = df_final["Release Date"].apply(extract_year)
df_final["Decade"] = df_final["Year_Extracted"].apply(lambda y: f"{int(y//10*10)}s" if not np.isnan(y) else pd.NA)


In [ ]:
# 8) Export balanced 60s–20s
decades_60_up = ['1960s','1970s','1980s','1990s','2000s','2010s','2020s']
df60 = df_final[df_final["Decade"].isin(decades_60_up)]
n60 = df60["Decade"].value_counts().min()
balanced60 = df60.groupby("Decade").sample(n=n60, random_state=42)
balanced60.to_csv("F-balanced_60s_up.csv", index=False)


In [ ]:
# 9) Export balanced 70s–20s
decades_70_up = ['1970s','1980s','1990s','2000s','2010s','2020s']
df70 = df_final[df_final["Decade"].isin(decades_70_up)]
n70 = df70["Decade"].value_counts().min()
balanced70 = df70.groupby("Decade").sample(n=n70, random_state=42)
balanced70.to_csv("F-balanced_70s_up.csv", index=False)


In [ ]:
# Cell 1: List, rename, and link final CSVs
import glob, os
from IPython.display import FileLink, display

csvs = glob.glob("F-*.csv") + glob.glob("Improved_Bucket_Accuracy_*.csv")
print("Found these CSVs in", os.getcwd(), ":\n", csvs)
for path in csvs:
    display(FileLink(path))

desired = {
    "Improved_Bucket_Accuracy_all_songs_raw_with_buckets.csv",
    "Improved_Bucket_Accuracy_balanced_60s_up.csv",
    "Improved_Bucket_Accuracy_balanced_70s_up.csv"
}
for fname in os.listdir('.'):
    if fname.startswith("Improved_Bucket_Accuracy_") and fname not in desired:
        original = "F-" + fname.replace("Improved_Bucket_Accuracy_", "", 1)
        os.rename(fname, original)
        print(f"Reverted {fname} → {original}")

print("\nFinal CSVs with improved buckets:")
for f in desired:
    if os.path.exists(f):
        print(f"- {f}")
print("Working directory:", os.getcwd())

for f in desired:
    display(FileLink(f))


In [ ]:
# Cell 2: Hold-out evaluation with hybrid SVM
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

csv_path = "Improved_Bucket_Accuracy_all_songs_raw_with_buckets.csv"
df = pd.read_csv(csv_path)
df['vector'] = df['vector'].apply(literal_eval)

has_meta = df[df['MetaGenres'].apply(lambda L: L != ["OtherMeta"])]
y = has_meta['Bucket'].values

tfidf = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(has_meta['Lyrics'].astype(str))
X_emb = csr_matrix(np.stack(has_meta['vector'].tolist()))
X = hstack([X_tfidf, X_emb])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2,
                                         random_state=42, stratify=y)
svc = LinearSVC(class_weight="balanced", max_iter=5000)
svc.fit(X_tr, y_tr)
y_pred = svc.predict(X_te)

print("Hold-out Accuracy:", accuracy_score(y_te, y_pred))
print(classification_report(y_te, y_pred, zero_division=0))


In [ ]:
# Cell 3: KNN cross-validation and hold-out test
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df['vector'] = df['vector'].apply(literal_eval)
has_meta = df[df['MetaGenres'].apply(lambda L: L != ["OtherMeta"])]

X = np.stack(has_meta['vector'].tolist())
y = has_meta['Bucket'].values

results = []
for k in [1, 3, 5, 7, 9, 11]:
    knn = KNeighborsClassifier(n_neighbors=k, metric="cosine")
    scores = cross_val_score(knn, X, y, cv=5, scoring="accuracy")
    results.append({'k': k, 'accuracy': scores.mean()})
print(pd.DataFrame(results))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
knn = KNeighborsClassifier(n_neighbors=5, metric="cosine")
knn.fit(X_train, y_train)
print(classification_report(y_test, knn.predict(X_test), zero_division=0))

knn = KNeighborsClassifier(n_neighbors=5, metric="cosine", weights="distance")
print("Weighted k-NN CV accuracy:", cross_val_score(knn, X, y, cv=5, scoring="accuracy").mean())


In [ ]:
# Cell 4: TF-IDF + Logistic Regression and TF-IDF+Embeddings+SVC
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from scipy import sparse
from sklearn.svm import LinearSVC

df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df['vector'] = df['vector'].apply(literal_eval)
has_meta = df[df['MetaGenres'].apply(lambda L: L != ["OtherMeta"])]

# TF-IDF + Logistic Regression
clf = make_pipeline(
    TfidfVectorizer(max_features=5000, stop_words="english"),
    LogisticRegression(max_iter=300, class_weight="balanced")
)
print("TF-IDF + LR CV accuracy:",
      cross_val_score(clf, has_meta['Lyrics'].astype(str),
                      has_meta['Bucket'], cv=5, scoring="accuracy").mean())

# TF-IDF + Embeddings + SVC
tfidf2 = TfidfVectorizer(max_features=3000, stop_words="english")
X_tfidf2 = tfidf2.fit_transform(has_meta['Lyrics'].astype(str))
X_emb2 = sparse.csr_matrix(np.stack(has_meta['vector'].tolist()))
X_combined = sparse.hstack([X_tfidf2, X_emb2])

svc2 = LinearSVC(class_weight="balanced", max_iter=5000)
print("TF-IDF + Embeddings + SVC CV accuracy:",
      cross_val_score(svc2, X_combined, has_meta['Bucket'], cv=5, scoring="accuracy").mean())


In [ ]:
# Cell 1: Hybrid SVM on regex‐decoded data
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

# 1) Load your data and parse the stored vectors
df = pd.read_csv("F-all_songs_with_regex_decade.csv")
df['vector'] = df['vector'].apply(literal_eval)

# 2) Keep only the songs with real metadata (ground truth)
has_meta = df[df['MetaGenres'].apply(lambda L: L != ["OtherMeta"])].copy()
y = has_meta['Bucket'].values

# 3) Build TF–IDF features from lyrics
tfidf = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(has_meta['Lyrics'].astype(str))

# 4) Build embedding features (your 384-dim vectors)
X_emb = csr_matrix(np.stack(has_meta['vector'].tolist()))

# 5) Combine into the hybrid feature matrix
X_hybrid = hstack([X_tfidf, X_emb])

# Option A: 5-fold CV accuracy
svc = LinearSVC(class_weight="balanced", max_iter=5000)
cv_scores = cross_val_score(svc, X_hybrid, y, cv=5, scoring="accuracy")
print(f"Hybrid SVM CV accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Option B: Hold-out split + full report
X_tr, X_te, y_tr, y_te = train_test_split(
    X_hybrid, y, test_size=0.2, random_state=42, stratify=y
)
svc.fit(X_tr, y_tr)
y_pred = svc.predict(X_te)
print("\nHybrid SVM hold-out classification report:\n")
print(classification_report(y_te, y_pred, zero_division=0))


In [ ]:
# Cell 2: Load PCA-refined data & split metadata/no-metadata
import pandas as pd
from ast import literal_eval

# Adjust filename if needed
df = pd.read_csv("F-all_songs_refined_with_PCA.csv")
df['vector'] = df['vector'].apply(literal_eval)

# If Decade not yet derived, do it now
if 'Decade' not in df.columns:
    df['Year_int'] = pd.to_datetime(df['Year'], errors='coerce').dt.year
    df['Decade']  = (df['Year_int'] // 10 * 10).astype('Int64').astype(str) + 's'
    df.drop(columns=['Year_int'], inplace=True)

print("Loaded DataFrame:", df.shape)
print("Columns:", df.columns.tolist())

# has_meta = songs with real Spotify genres  
has_meta = df[df['MetaGenres'].apply(lambda L: L != ["OtherMeta"])].copy()
# no_meta  = songs we will predict via the hybrid SVM
no_meta  = df[df['MetaGenres'].apply(lambda L: L == ["OtherMeta"])].copy()

print("has_meta rows:", has_meta.shape[0])
print("no_meta rows: ", no_meta.shape[0])


In [ ]:
# Cell 3: Retrain hybrid SVM on metadata-rich PCA data, predict missing, export
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import LinearSVC

# 1) TF–IDF on has_meta lyrics
tfidf2 = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1,2))
X_tfidf2 = tfidf2.fit_transform(has_meta['Lyrics'].astype(str))

# 2) Embedding matrix from precomputed vectors
X_emb2 = csr_matrix(np.stack(has_meta['vector'].tolist()))

# 3) Hybrid feature matrix
X_hybrid2 = hstack([X_tfidf2, X_emb2])

# 4) Train SVM
y2 = has_meta['Bucket'].values
svc2 = LinearSVC(class_weight="balanced", max_iter=5000)
svc2.fit(X_hybrid2, y2)
print("Hybrid SVM trained on metadata-rich songs")

# 5) Predict buckets for no_meta
X_tfidf_no = tfidf2.transform(no_meta['Lyrics'].astype(str))
X_emb_no   = csr_matrix(np.stack(no_meta['vector'].tolist()))
X_hybrid_no = hstack([X_tfidf_no, X_emb_no])

preds_no = svc2.predict(X_hybrid_no)
no_meta['Bucket'] = preds_no

# 6) Merge back and export
df.loc[no_meta.index, 'Bucket'] = no_meta['Bucket']
df.to_csv("F-all_songs_raw_with_buckets.csv", index=False)
print("Exported F-all_songs_raw_with_buckets.csv")


In [ ]:
# Cell 4: Extract year from Release Date, assign Decade, export balanced sets
import pandas as pd
import numpy as np
import re

# 1) Load final CSV
df_final = pd.read_csv("F-all_songs_raw_with_buckets.csv")

# 2) Extract first 4-digit year
def extract_year(s):
    m = re.search(r"(\d{4})", str(s))
    return int(m.group(1)) if m else np.nan

df_final['Year_Extracted'] = df_final['Release Date'].apply(extract_year)

# 3) Map to decades
df_final['Decade'] = df_final['Year_Extracted'].apply(
    lambda y: f"{int(y//10*10)}s" if not np.isnan(y) else "Unknown"
)

print(df_final['Decade'].value_counts(dropna=False).sort_index())

# 4) Export balanced 60s–2020s
dec60 = ['1960s','1970s','1980s','1990s','2000s','2010s','2020s']
df60 = df_final[df_final['Decade'].isin(dec60)]
n60 = df60['Decade'].value_counts().min()
balanced60 = df60.groupby('Decade').sample(n=n60, random_state=42)
balanced60.to_csv("F-balanced_60s_up.csv", index=False)

# 5) Export balanced 70s–2020s
dec70 = ['1970s','1980s','1990s','2000s','2010s','2020s']
df70 = df_final[df_final['Decade'].isin(dec70)]
n70 = df70['Decade'].value_counts().min()
balanced70 = df70.groupby('Decade').sample(n=n70, random_state=42)
balanced70.to_csv("F-balanced_70s_up.csv", index=False)


In [ ]:
# Cell 5: List & link your final CSVs
import glob, os
from IPython.display import FileLink, display

csvs = glob.glob("F-*.csv") + glob.glob("Improved_Bucket_Accuracy_*.csv")
print("CSV files found:", csvs)
for c in csvs:
    display(FileLink(c))

desired = {
    "Improved_Bucket_Accuracy_all_songs_raw_with_buckets.csv",
    "Improved_Bucket_Accuracy_balanced_60s_up.csv",
    "Improved_Bucket_Accuracy_balanced_70s_up.csv"
}
for fname in os.listdir('.'):
    if fname.startswith("Improved_Bucket_Accuracy_") and fname not in desired:
        os.rename(fname, "F-" + fname.replace("Improved_Bucket_Accuracy_", "", 1))
